In [16]:
import pandas as pd
import json

from tqdm import tqdm
from transformers import AutoTokenizer

In [18]:
bronze_db = pd.read_csv("../data/train.csv")
bronze_db

,id,text,entities
0,582dab45ea473d07323e3c6a0415425c037e63b5a7ab0a...,"Зрозуміло , що український бізнес почав викори...","[{""label"": ""MISC"", ""text"": ""\u041a\u0421\u0412..."
1,df0933559918588668cb4219c80908b33cdef041dce468...,\nДолучилися до заходу заступник голови райдер...,"[{""label"": ""JOB"", ""text"": ""\u0437\u0430\u0441\..."
2,6b66cabd04bce970de47e4dbef5c1c0219591d296d4285...,"У неділю , 4 березня , у Чернівцях у ліцеї № 4...","[{""label"": ""DATE"", ""text"": ""4 \u0431\u0435\u04..."
3,3ca90ad4d912c780b305407035af5450c5136d52d11497...,"Міжнародний благодійний фонд « Покуття » , фун...","[{""label"": ""ORG"", ""text"": ""\u041c\u0456\u0436\..."
4,27a06d92f27fdb30ffc4a13cafd98d81ae1cc7f436b865...,"Цю прогалину , життєпис відомого колись на всю...","[{""label"": ""JOB"", ""text"": ""\u0444\u043e\u0442\..."
...,...,...,...
386,84a947547d6ba2e726e937bdc3186d7f242dd24d835b41...,Філія «УМГ «Львівтрансгаз» ПАТ «Укртрансгаз» 2...,"[{""label"": ""ORG"", ""text"": ""\u0423\u041c\u0413 ..."
387,9340ab70312c12f9e3da65b13c31d7cccb234cac3973a5...,Сім'я судді Апеляційного суду м. Києва Дениса ...,"[{""label"": ""JOB"", ""text"": ""\u0441\u0443\u0434\..."
388,8e28367fa94e8dd2f96a2a75eeb74767a382092bdad77d...,Департамент капітального будівництва Одеської ...,"[{""label"": ""ORG"", ""text"": ""\u0414\u0435\u043f\..."
389,3cbbc257a6d296840d03e4ec7518a4f1b2c691458072d4...,Компанія з орбіти нардепа від ОПЗЖ Вадима Стол...,"[{""label"": ""JOB"", ""text"": ""\u043d\u0430\u0440\..."


In [5]:
checkpoint = "CohereForAI/aya-23-8b"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
MAX_TOKENS = 500

In [8]:
def parse_entities(entities_str):
    try:
        return json.loads(entities_str)
    except json.JSONDecodeError:
        return []

bronze_db['entities'] = bronze_db['entities'].apply(parse_entities)

In [9]:
def split_text_and_entities(row, tokenizer, max_tokens=500):
    text = row['text']
    entities = row['entities']
    original_id = row['id']

    # Split text into lines based on newlines
    lines = text.split('\n')

    chunks = []
    current_chunk = ""
    current_entities = []

    for line in lines:
        # Tentatively add the line to the current chunk
        tentative_chunk = f"{current_chunk}\n{line}" if current_chunk else line
        tokens = tokenizer.encode(tentative_chunk, add_special_tokens=False)

        if len(tokens) <= max_tokens:
            current_chunk = tentative_chunk
        else:
            if current_chunk:
                # Finalize the current chunk
                chunks.append((current_chunk, current_entities.copy()))
                current_chunk = line  # Start a new chunk with the current line
                current_entities = []
            else:
                # Handle the case where a single line exceeds max_tokens
                # Strategy: Truncate the line to fit
                truncated_tokens = tokenizer.encode(line, add_special_tokens=False)[:max_tokens]
                truncated_text = tokenizer.decode(truncated_tokens, clean_up_tokenization_spaces=True)
                chunks.append((truncated_text, []))
                current_chunk = ""

        # Assign entities to the current chunk
        for entity in entities:
            entity_text = entity['text']
            if entity_text in line and entity not in current_entities:
                current_entities.append(entity)

    # Append any remaining chunk
    if current_chunk:
        chunks.append((current_chunk, current_entities.copy()))

    # Create split rows maintaining the original id
    split_rows = []
    for chunk_text, chunk_entities in chunks:
        split_rows.append({
            'id': original_id,  # Keep the original id
            'text': chunk_text,
            'entities': json.dumps(chunk_entities, ensure_ascii=False)
        })

    return split_rows

# Step 5: Apply the Splitting Function to the DataFrame
split_data = []

for _, row in tqdm(bronze_db.iterrows()):
    split_rows = split_text_and_entities(row, tokenizer, MAX_TOKENS)
    split_data.extend(split_rows)

# Create a new DataFrame from the split data
silver_df = pd.DataFrame(split_data)
silver_df

391it [00:08, 47.12it/s] 


,id,text,entities
0,582dab45ea473d07323e3c6a0415425c037e63b5a7ab0a...,"Зрозуміло , що український бізнес почав викори...","[{""label"": ""MISC"", ""text"": ""КСВ""}, {""label"": ""..."
1,582dab45ea473d07323e3c6a0415425c037e63b5a7ab0a...,Тому підтримка ЗСУ стала для підприємців добро...,"[{""label"": ""ORG"", ""text"": ""ЗСУ""}, {""label"": ""L..."
2,582dab45ea473d07323e3c6a0415425c037e63b5a7ab0a...,"Цікаво також , що дітям розповідають , які вим...",[]
3,df0933559918588668cb4219c80908b33cdef041dce468...,Долучилися до заходу заступник голови райдержа...,"[{""label"": ""JOB"", ""text"": ""заступник голови ра..."
4,df0933559918588668cb4219c80908b33cdef041dce468...,Заступник голови райдержадміністрації наголоси...,"[{""label"": ""ORG"", ""text"": ""РДА""}, {""label"": ""J..."
...,...,...,...
1074,8e28367fa94e8dd2f96a2a75eeb74767a382092bdad77d...,"Нагадаємо, що «Ремерцентр» раніше також стягну...","[{""label"": ""ORG"", ""text"": ""Північна""}, {""label..."
1075,3cbbc257a6d296840d03e4ec7518a4f1b2c691458072d4...,Компанія з орбіти нардепа від ОПЗЖ Вадима Стол...,"[{""label"": ""JOB"", ""text"": ""нардепа""}, {""label""..."
1076,3cbbc257a6d296840d03e4ec7518a4f1b2c691458072d4...,Містобудівні умови та обмеження на будівництво...,"[{""label"": ""DATE"", ""text"": ""2017 року""}, {""lab..."
1077,6ae02040ca747c93b4e534d98c3084dda65d56c8b43d86...,Вищий господарський суд відмовився задовольнит...,"[{""label"": ""ORG"", ""text"": ""Вищий господарський..."


In [15]:
silver_df.to_csv("../data/silver.csv", index=False)

In [19]:
import hvplot.pandas

silver_df.loc[:, "entities_tokens"] = silver_df.loc[:, "entities"].apply(lambda x: len(tokenizer.tokenize(x)))
silver_df.loc[:, "entities_tokens"].hvplot.kde()

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


:Distribution   [entities_tokens]   (Density)